# Ejercicio usando PySpark

## Importar librerías

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import plot_confusion_matrix
from sklearn import metrics, model_selection, naive_bayes 
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from collections import Counter, defaultdict

## Carga y exploración inicial de base de datos

In [2]:
# Cargar bases de datos
df = spark.read.load('abfss://entrenamiento@datalakecomfama.dfs.core.windows.net/AndresTQ/pokemon.csv', 
format='csv', header=True)

In [3]:
# Revisamos los primeros registros
# df.head() # Forma en python, pero no se ve bien
display(df.limit(10))

In [4]:
# Revisamos el tamaño de nuestra base de datos
print(df.count(),len(df.columns))

In [6]:
# Validamos que las funciones normales de pandas no funcionan acá
df.shape

In [5]:
# Revisamos el listado de variables 
df.columns

In [6]:
# Recortamos el nombre de las variables 'against' para reducir espacios en encabezados
df = df.replace("against", "ag")

In [7]:
# Revisamos el cambio
display(df.limit(10)) 

# Ejercicio usando Python
## Pasando la base de datos a pandas

In [8]:
# Convertir archivo spark a pandas
datos = df.toPandas()

## Exploración inicial de los datos

In [9]:
# Revisamos el nuevo tamaño de nuestros datos. Validamos que ya si ejecutan las funciones de pandas
datos.shape



Esta es una base de datos que cuenta con 801 registros y 41 columnas/variables, donde cada registro hace alusión a un pokemon en particular, y las variables son las características asociadas a cada uno de ellos.

In [10]:
# Codigo para que se desplieguen todas las columnas del dataframe
pd.set_option('display.max_columns', None)
# Revisamos los primeros registros
datos.head()

In [11]:
# Revisamos las variables
list(datos.columns.values)

Como se había mencionado, las variables de este dataset hacen referencia a las características de cada uno de los pokemon, como su tipo (planta, fuego, eléctrico, etc), habilidades (velocidad, defensa, ataque, etc) y resistencias contra otros tipos. Así mismo se indica el nombre del pokemon, si es legendario o no, su número en la pokedex y la generación a la que pertenece

## Actividad

La variable a predecir será el tipo 1 (type1). Haremos un ejercicio tanto de aprendizaje supervisado como no supervisado.

Para este ejercicio haremos uso de las variables de habilidades (ataque, defensa, velocidad, etc.) y resistencias contra otros tipos. Descartaremos la generación, número en pokedex, si es legendario o no, la segunda tipología (type2).

## Ajuste previo en la tipología de los datos

In [12]:
# Revisamos la tipología de los datos
datos.dtypes

In [13]:
# Convertimos a numéricas nuestras variables
cols = datos.columns.drop('type1')
datos[cols] = datos[cols].apply(pd.to_numeric, errors='coerce')

In [14]:
# Revisamos nuevamente la tipología para valdiar el ajuste
datos.dtypes

## Preparación y limpieza de datos

In [15]:
# Recortamos el nombre de las variables 'against' para reducir espacios en encabezados
datos.columns = datos.columns.str.replace("against", "ag")

### Selección de variables

In [16]:
# Vamos a elegir variables que usaremos para nuestro ejercicio 
datos_modelo = datos.loc[:, ['ag_bug', 'ag_dark', 'ag_dragon', 'ag_electric', 'ag_fairy', 'ag_fight', 'ag_fire', 
                             'ag_flying', 'ag_ghost', 'ag_grass', 'ag_ground', 'ag_ice', 'ag_normal', 'ag_poison', 
                             'ag_psychic', 'ag_rock', 'ag_steel', 'ag_water', 'attack', 'defense', 'height_m', 
                             'hp', 'sp_attack', 'sp_defense', 'speed', 'weight_kg', 'type1']]

### Valores faltantes

In [17]:
# Revisamos la cantidad de NA en cada variable
datos_modelo.isna().sum()


Tenemos 20 registros con NA en el peso y 20 registros con NA en la altura. Alternativas:

* Eliminar
* Imputar

En este caso, pensar en eliminar podría ser contraproducente dado que tenemos muy pocos registros para entrenar el modelo. Por esta razón procedremos a imputar nuestros datos. El método a utilizar será la imputación de características multivariadas, la cual imputa una variable como una función de las otras variables. Este estimador todavía está en proceso experimental, y pertenece a la librería de scikit learn.

In [18]:
# Creamos objeto para almacenar modelo de imputación
imp = IterativeImputer(max_iter=10, random_state=0)
# Entrenamos el modelo de imputación con los datos de nuestro dataset
imp.fit(datos_modelo.iloc[:,0:-1])

In [19]:
# Aplicamos el modelo de imputación en nuestros datos
datos_modelo.iloc[:,0:-1] = imp.transform(datos_modelo.iloc[:,0:-1])

In [20]:
# Revisamos que ya no tengamos presencia de valores faltantes
datos_modelo.isna().sum()

In [21]:
# Validamos que no se hayan eliminado registros
datos_modelo.shape

## Análisis exploratorio

### Estadísticos generales

In [22]:
# Estadísticos de nuestras variables
datos_modelo.describe()

De estos descriptivos resaltan dos grupos:

* Las variables de resistencia que, en su gran mayoría, están entre 0/0.25 y 4, sirviendo como escalafón para indicar qué tanto daño reciben de los otros tipos de pokemon, siendo 0 inmunidad y 4 es que recibe el daño multiplicado por 4.
* Las demás variables registran unos rangos más amplios y diversos (amerita estandarizar los datos más adelante). En promedio, un pokemon tendría las siguientes características:
    * Peso de 61 kg
    * Altura de 1.16 metros
    * 68 puntos de vida
    * Ataque de 77 puntos, defensa de 73 y velocidad de 66.

Llama la atención el mínimo en el peso puesto que es negativo, lo cual no tendría sentido desde la lógica de la variable en sí. Valdría la pena analizarlo para determinar si se debe eliminar o no, pero no lo haremos en este caso.

### Registros por tipología

In [23]:
# Revisamos la cantidad de pokemones que hay por tipo
sns.countplot(x = "type1", data = datos_modelo, order = datos_modelo['type1'].value_counts().index)
plt.xticks(rotation = 45)
plt.xlabel("Tipo")
plt.ylabel("Cantidad")
plt.title("Cantidad de pokemon por Tipo")
plt.show()
#datos.loc[:,'type1'].value_counts()

Se observa que la mayoría de pokemones son de tipo agua, seguidos de los normales y planta. El top 5 lo cierran los tipo insecto y psíquico.

### Características promedio por tipología

In [24]:
# Revisamos por tipologpia el promedio de cada una de las variables
datos_modelo.groupby('type1').aggregate({'speed': 'mean',
                                         'attack': 'mean',
                                         'defense': 'mean',
                                         'sp_attack': 'mean', 
                                         'sp_defense': 'mean',
                                         'hp': 'mean', 
                                         'height_m': 'mean',
                                         'weight_kg': 'mean', 
                                         'sp_defense': 'mean',
                                         'ag_bug': 'mean', 
                                         'ag_dark': 'mean',
                                         'ag_dragon': 'mean', 
                                         'ag_electric': 'mean', 
                                         'ag_fairy': 'mean',
                                         'ag_fight': 'mean', 
                                         'ag_fire': 'mean', 
                                         'ag_flying': 'mean',
                                         'ag_ghost': 'mean', 
                                         'ag_grass': 'mean', 
                                         'ag_ground': 'mean',
                                         'ag_ice': 'mean', 
                                         'ag_normal': 'mean', 
                                         'ag_poison': 'mean',
                                         'ag_psychic': 'mean', 
                                         'ag_rock': 'mean', 
                                         'ag_steel': 'mean',
                                         'ag_water': 'mean'}).reset_index().round(2)

Del cuadro anterior se pueden ir haciendo algunas observaciones que darían luces sobre las características que permitirán diferenciar cada tipo de pokemon en los modelos posteriores. Algunos ejemplos de estas diferencias serían los siguientes:

* Las resistencias de cada tipo vs los otros probablemente serán relevantes. Ejemplo, se observa que los tipo agua, roca y arena son menos resistentes contra los tipo planta. Por otro lado, los tipo normal son completamente inmunes contra los fantasmas.
* Los tipo voladores son de lejos los de mayor velocidad promedio.
* En cuanto a defensa, los de hierro tienen una gran diferenciación siendo los de mayor valor en promedio.

### Gráfico de correlación

In [25]:
# Calculamos correlaciones
grafica_correlacion = datos_modelo.corr()

In [26]:
# Graficamos la correlación
plt.figure(figsize=(22, 12))
sns.heatmap(grafica_correlacion, annot = True, linewidths = 0.5)
plt.show()

De este gráfico se confirman algunas de las observaciones hechas previamente:

* Las resistencias vs otros tipos podrán ser relevantes:
          
  * Estas suelen tener relaciones relativamente fuertes entre sí, ya sean positivas o negativas. Ejemplo: La correlación entre las resistencias contra los tipo planta y contra los tipo fuego es de -0.46. Esto indicaría que aquellos que son fuertes contra los planta normalmente son débiles contra los fuego, o viceversa (Ejemplo los de agua). 
  
* Las características de los pokemon presentan relaciones relativamente fuertes y positivas entre sí. Los de mayor ataque suelen también tener buena defensa. Los de mayor peso y altura suelen tener más puntos de vida. 

## Preparación de los datos. Entrenamiento y testeo.

#### Separamos las variables explicativas (de insumo) de la variable a explicar (objetivo)

In [27]:
# Separamos los variables explicativas de la variable objetivo
X = datos_modelo.iloc[:,0:-1]
y = datos_modelo.iloc[:,-1]

In [28]:
# Plantamos semilla para garantizar reproducibilidad
np.random.seed(110)
# Dividimos la base con una proporción 80 % - 20 %
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#### Revisamos los tamaños de nuestras nuevas bases

In [29]:
# Base de entrenamiento
print('El tamaño de nuestra base de entrenamiento de las variables insumo es:')
X_train.shape

In [30]:
# Base de testeo
print('El tamaño de nuestra base de testeo de las variables insumo es:')
X_test.shape

#### Validamos que se sigan conservando las proporciones de muestras

In [31]:
# Revisamos la cantidad de pokemones que hay por tipo
sns.countplot(x = "type1",data = pd.DataFrame(y_train), order = pd.DataFrame(y_train)['type1'].value_counts().index)
plt.xticks(rotation = 45)
plt.xlabel("Tipo")
plt.ylabel("Cantidad")
plt.title("Cantidad de pokemon por Tipo")
plt.show()

Se conservan las proporciones observadas anteriormente, donde los de mayor representatividad siguen siendo los acuáticos, normales, planta, insecto y psíquico respectivamente.

#### Estandarización de los datos

En este punto se pretende estandarizar las bases de datos de nuestras variables de insumo. Esta estandarización se realiza a nivel de cada variable, donde se resta el promedio y se divide entre la desviación estándar de la respectiva variable, con lo cual se busca que la distribución de estas sea más o menos similar a la de una distribución normal (media 0, desviación estándar 1)

In [32]:
# Procedemos a estandarizar los datos de las variables "x". Se estandariza con el conjunto de entrenamiento.
estandarizacion = StandardScaler()
estandarizacion.fit(X_train)

# Con los parámetros de estandarización entrenados, se realiza la estandarización los dos conjuntos de datos
X_train_est = estandarizacion.transform(X_train)
X_test_est = estandarizacion.transform(X_test)

## Ejercicio aprendizaje supervisado

De acuerdo a lo propuesto en la actividad, plantearemos tres modelos que buscan predecir el tipo de pokemon basado en el resto de variables observadas. Se proponen los siguientes modelos:

* **Naive Bayes** 
* **Regresión Logística**
* **Random Forest**

### Naive Bayes

#### Entrenamiento del modelo

In [33]:
# Especificamos el modelo a usar
nv = naive_bayes.GaussianNB()

# Entrenamos el modelo
nv.fit(X_train_est, y_train)

#### Predicción del modelo en los datos de testeo

In [34]:
# Hacemos las predicciones sobre el conjunto de testeo
y_pred_nv = nv.predict(X_test_est)

#### Validación de resultados

In [35]:
# Calculamos la precisión del clasificador
acc_nv = metrics.accuracy_score(y_test, y_pred_nv)
print ("Precisión del modelo naive bayes: %.2f " %(acc_nv*100) )

# Mostramos la cantidad de muestras mal clasificadas 
print('Las muestras mal clasificadas en el modelo naive bayes fueron %d' % (y_test != y_pred_nv).sum())

# Visualizamos reporte con los resultados generales
print(classification_report(y_test, y_pred_nv))

#### Matriz de confusión

In [36]:
# Ajustamos tamaño de la matriz
fig, ax = plt.subplots(figsize=(14, 12))
# Construmios el gráfico
plot_confusion_matrix(nv, X_test_est, y_test, cmap=plt.cm.Blues, ax = ax)  
# Visualizamos
plt.show()

### Regresión Logística

#### Entrenamiento del modelo

In [37]:
# Especificamos el modelo a usar
reg_logistica = LogisticRegression(C = 1000, random_state = 0, max_iter = 500)

# Entrenamos el modelo
reg_logistica.fit(X_train_est, y_train)

#### Predicción del modelo en los datos de testeo

In [38]:
# Hacemos las predicciones sobre el conjunto de testeo
y_pred_reglog = reg_logistica.predict(X_test_est)

#### Validación de resultados

In [39]:
# Calculamos la precisión del clasificador
acc_nv = metrics.accuracy_score(y_test, y_pred_reglog)
print ("Precisión del modelo de regresión logística: %.2f " %(acc_nv*100) )

# Mostramos la cantidad de muestras mal clasificadas 
print('Las muestras mal clasificadas en el modelo de regresión logística fueron %d' % (y_test != y_pred_reglog).sum())

# Visualizamos reporte con los resultados generales
print(classification_report(y_test, y_pred_reglog))

#### Matriz de confusión

In [40]:
# Ajustamos tamaño de la matriz
fig, ax = plt.subplots(figsize=(14, 12))
# Construmios el gráfico
plot_confusion_matrix(reg_logistica, X_test_est, y_test, cmap=plt.cm.Blues, ax = ax)  
# Visualizamos
plt.show()

### Random Forest

#### Entrenamiento del modelo

In [41]:
# Sembramos semilla
np.random.seed(100)
# Especificamos el modelo a usar
random_forest = RandomForestClassifier(n_estimators=100)

# Entrenamos el modelo
random_forest.fit(X_train_est, y_train)

#### Predicción del modelo en los datos de testeo

In [42]:
# Hacemos las predicciones sobre el conjunto de testeo
y_pred_rf = random_forest.predict(X_test_est)

#### Validación de resultados

In [43]:
# Calculamos la precisión del clasificador
acc = metrics.accuracy_score(y_test, y_pred_rf)
print ("Precisión del modelo random forest: %.2f " %(acc*100) )

# Mostramos la cantidad de muestras mal clasificadas 
print('Las muestras mal clasificadas fueron %d' % (y_test != y_pred_rf).sum())

# Visualizamos reporte con los resultados generales
print(classification_report(y_test, y_pred_rf))

#### Matriz de confusión

In [44]:
# Ajustamos tamaño de la matriz
fig, ax = plt.subplots(figsize=(14, 12))
# Construmios el gráfico
plot_confusion_matrix(random_forest, X_test_est, y_test, cmap=plt.cm.Blues, ax = ax)  
# Visualizamos
plt.show()

### Comparación entre modelos

Al analizar los tres modelos se encuentra que:

* En términos de **accuracy** el mejor modelo es el de **Regresión Logística con un 91.30%**, seguido del **Random Forest con 90.06%**, y el de peor desempeño fue el **Naive Bayes con 64.60%**.
* En concordancia con lo anterior se tiene que, de los 161 registros en la base de testeo, el modelo de Regresión Logística tuvo una mala clasificación en 14 casos, el Random Forest en 16 y el Naive Bayes en 57.
* En los tres modelos suele haber algunos errores en el pronóstico de los tipo dragón, hierro, roca y psíquico. Esto medido a través del F1-score. Probablemente sea necesario alguna(s) variable(s) adicional(es) que permitan mejorar el pronóstico para estos.

Por lo anterior, para este ejercicio se decide que el modelo a emplear es el de *Regresión Logística*.

#### Notas

* Para este ejercicio no se realizó un ejercicio riguroso de tuning. Esto quiere decir que no se hicieron grandes ajustes a los parámetros de los algoritmos. Tuning es un proceso que se emplea para mejorar el desempeño de cada uno de los modelos.
* Después de estos resultados el siguiente paso sería desplegar este modelo en un entorno de producción.

## Ejercicio con aprendizaje no supervisado

En este caso omitiremos la variable obejtivo ***Y***, y únicamente a partir del conjunto de datos ***X*** intentaremos consturir clusters. En esencia este modelo desconoce la existencia de los tipos de pokemon, por lo cual es probable que el número de clusters determinados sea distinto a la cantidad de tipos originales. Para este ejercicio solamente utilizaremos un modelo:

* **K-Means:** Es uno de los modelos de clusterización más populares, en el cual se determinan "centroides" que representan cada uno de los clusters definidos. La definición de estos centroides se da al tratar de minimizar la distancia de los registros al interior de los clusters.

#### Determinación de centroides óptimos

In [45]:
# Creamos matriz para almacenar los datos de las distancias por cada centroide
suma_distancias = []
# Establecemos un rango de centroides
K = range(1, 25)
# Ejecutamos ciclo for para 
for i in K:
    kmeans_model = KMeans(n_clusters = i, random_state = 0).fit(X_train_est)
    suma_distancias.append(kmeans_model.inertia_) #.inertia_ calcula la suma de las distancias al cuadrado de todos los puntos vs el centroides más cercano

In [46]:
# Graficamos 
plt.plot(K, suma_distancias, 'bx-')
plt.xlabel('k')
plt.ylabel('Suma de distancias al cuadrado')
plt.title('Método Elbow - K óptimo')
plt.show()

De acuerdo al gráfico anterior, se podría decir que el número óptimo de clusters es **14**. A partir de este cluster la disminución en la distancia tiende a ser menor, con lo que se nota que se empieza a volver más horizontal (punto del codo).

#### Construimos el modelo con los centroides definidos

In [47]:
# Entrenamos el modelo kmeans con los 14 centroides definidos
kmeans = KMeans(n_clusters = 14, random_state = 0).fit(X_train_est)

#### Revisamos las características de los clusters

In [48]:
# Revisamos las características de los centroides definidos en el modelo
caracteristicas_clusters = pd.DataFrame(estandarizacion.inverse_transform(kmeans.cluster_centers_).round(2), columns = X.columns.values)
caracteristicas_clusters

En este punto corresponde hacer un análisi descriptivo con las características asociadas a cada uno de los clusters hallados. Dado que son muchos clusters haremos el análisis únicamente con dos clusters:

* El **cluster 9** se caracteriza por abarcar a los pokemon de mayor peso, mayor estatura y ser el segundo con mejores puntos de defensa. En cuanto a su resistencia frente a los otros tipos se tiene que su mayor debilidad es frente a los tipos hielo, mientras que contra los tipo fuego son más resistentes. Sin embargo no son diferencias muy marcadas. Se podría clasificar a este grupo como los "más grandes"
* El **cluster 3** se caracteriza porque es donde se ecuentran los pokemones con debilidad frente a un mayor número de tipologías. En efecto, los pokemones pertenecientes a este cluster suelen ser más débiles contra 5 tipos de pokemon (insecto, fuego, volador, hielo y veneno). Se podría clasificar a este grupo como los "más débiles".

In [49]:
# Revisamos la cantidad de registros por cluster
Counter(kmeans.labels_)

#### Pronosticamos sobre la base de testeo

In [50]:
kmeans.predict(X_test_est)

#### Notas

A diferencia del modelo de aprendizaje supervisado, en este caso no contamos con una métrica que nos ayude a definir qué tan bien quedó nuestra clusterización. La determinación de si este es un buen modelo surge del análisis descriptivo de los clusters. En efecto, al revisar los segmentos hallados debemos plantearnos si estos hacen sentido en el contexto que estemos analizando. 